请先阅读完作业要求后，完成本次作业。

在本次作业里，我们将要实现以下内容：
1. 建立一个针对数学问题的 zero-shot 评测集
2. 使用一个更强推理模型蒸馏得到的推理轨迹进行 SFT 训练，得到一个参数更小的推理模型，并测试其效果
3. 使用专家迭代（EI）算法，训练得到一个推理模型，并测试其效果
4. 使用 GRPO 算法，训练得到一个推理模型，并测试其效果
5. 针对 GRPO 算法，进行一系列调参实验，探索不同策略下对模型性能的影响

# 环境配置

再开始正式的训练之前，需要先搭建好一个能正常运行下面所有代码的环境。如同之前的作业，我们使用 `uv` 来管理依赖。

1. 先安装除 flash-attn 之外的所有包，然后再安装全部包（因为 flash-attn 比较特殊）：

    ```
    uv sync --no-install-package flash-attn
    #source .venv/bin/activate
    uv sync
    ```

2. 运行单元测试：

    ``` sh
    uv run pytest
    ```

# 一、 建立针对数学问题的评估基准集

首先，我们需要建立一个能对数学问题进行评估的评测集，并将其应用于基线模型，评估其 zero-shot 数学能力。我们此处的配置如下：

- **baseline model**: `Qwen2.5-Math-1.5B`
- **benchmark**: `GSM8K`（官方文档里评测和带有推理轨迹的训练使用的都是 `MATH 12K`，但由于版权限制并没有在`data/`目录下给出此数据，推荐使用一些其他的数据进行替代。但是笔者在查询了 MATH 12K的规定后，发现其使用的是宽松的 MIT license 开源协议，但是该数据不带有推理轨迹，因此只能用来评测不能用作后续训练，我们这里使用了 GSM8K 数据集来替代 MATH 12K 用作评测。如果有小伙伴想要使用该数据可自行下载 [MATH 12K](https://huggingface.co/datasets/qwedsacf/competition_math)。

## 1.1 评估基准集的设置

建立数学推理评估基准集是整个作业的基础。我们需要：

1. **选择合适的基准数据集**：由于 MATH 12K 数据集版权限制，我们使用 GSM8K 数据集作为替代。GSM8K (Grade School Math 8K) 是一个小学数学问题数据集，包含 8K 个问题，每个问题都有详细的推理过程和最终答案，问题难度适中，适合测试基础数学推理能力
2. **选择合适的提示格式**：使用 R1-Zero 提示格式，要求模型进行思维链推理
3. **定义评估指标**：使用格式奖励和答案准确率来评估模型性能
4. **使用高效的推理引擎**：使用 vLLM 进行批量推理，提高评估效率

### 实现逻辑

我们的评估流程包含以下几个关键步骤：

1. **数据加载与预处理**：加载 GSM8K 数据集，支持 JSONL 和 JSON 数组两种格式
2. **提示格式化**：将原始问题转换为 R1-Zero 格式的提示
3. **批量推理**：使用 vLLM 高效生成模型响应
4. **结果评估**：使用奖励函数计算格式和答案准确率
5. **统计分析**：分析不同组合的性能表现
6. **结果保存**

现在让我们逐步实现这些功能。

## 1.2 R1-Zero 提示格式

R1-Zero 提示格式是 DeepSeek R1 模型使用的推理提示格式，它要求模型：

1. 在 `<think>` 和 `</think>` 标签之间进行推理过程
2. 在 `<answer>` 和 `</answer>` 标签之间给出最终答案
3. 进行逐步推理，而不是直接给出答案

这种格式便于我们解析模型的输出，提取推理过程和最终答案。

### 实现细节

在 `evaluate_math.py` 中，我们实现了两个核心函数来处理提示：

1. **`load_r1_zero_prompt()`**：从文件加载 R1-Zero 提示模板
2. **`format_prompt()`**：将数学问题插入到提示模板中

提示模板文件位于 `cs336_alignment/prompts/r1_zero.prompt`，包含完整的对话格式。

In [1]:
# 加载 R1-Zero 提示模板
def load_r1_zero_prompt(prompt_file_path: str) -> str:
    """从文件加载 R1-Zero 提示模板
    
    这个函数的实现逻辑：
    1. 使用 UTF-8 编码打开文件
    2. 读取完整文件内容
    3. 使用 strip() 移除首尾空白字符
    
    为什么需要 strip()？
    - 文件末尾可能有换行符或其他空白字符
    - 移除这些字符可以确保模板格式的整洁性
    """
    with open(prompt_file_path, "r", encoding="utf-8") as f:
        return f.read().strip()

# 读取提示模板
PROMPT_PATH = "cs336_alignment/prompts/r1_zero.prompt"
try:
    r1_zero_template = load_r1_zero_prompt(PROMPT_PATH)
    print("R1-Zero 提示模板：")
    print("=" * 50)
    print(r1_zero_template)
    print("=" * 50)
except FileNotFoundError:
    print(f"提示模板文件未找到: {PROMPT_PATH}")
    print("请确保文件路径正确，或创建相应的提示文件")

R1-Zero 提示模板：
A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer. The reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>.
User: {question}
Assistant: <think>


In [3]:
# 格式化数学问题为 R1-Zero 提示
def format_prompt(question: str, prompt_template: str) -> str:
    """将数学问题格式化为 R1-Zero 提示
    
    这个函数的实现逻辑：
    1. 使用字符串的 format() 方法
    2. 将问题插入到模板中的 {question} 占位符处
    3. 返回格式化后的完整提示
    
    这种方法的好处：
    - 模板和内容分离，便于维护
    - 支持复杂的提示格式
    - 避免字符串拼接的错误
    """
    return prompt_template.format(question=question)

# 示例问题
sample_question = "小明有5个苹果，他又买了3个，现在他有多少个苹果？"

# 只有在成功加载模板后才进行格式化
if 'r1_zero_template' in locals():
    formatted_prompt = format_prompt(sample_question, r1_zero_template)

    print("原始问题：")
    print(sample_question)
    print("\n格式化后的提示：")
    print("=" * 50)
    print(formatted_prompt)
    print("=" * 50)
    
else:
    print("无法演示格式化，因为提示模板未加载")

原始问题：
小明有5个苹果，他又买了3个，现在他有多少个苹果？

格式化后的提示：
A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer. The reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>.
User: 小明有5个苹果，他又买了3个，现在他有多少个苹果？
Assistant: <think>


## 1.3 奖励函数设计

我们使用两个维度的奖励来评估模型的性能：

1. **格式奖励 (Format Reward)**：检查模型输出是否符合 R1-Zero 格式要求，格式正确得 1 分，否则得 0 分
   - 要求输出包含 `<think>` 和 `</think>` 标签
   - 要求输出包含 `<answer>` 和 `</answer>` 标签

2. **答案奖励 (Answer Reward)**：检查模型给出的答案是否正确，答案正确得 1 分，否则得 0 分
   - 从 `<answer>` 标签中提取答案
   - 与标准答案进行数学等价性比较

### 奖励函数的实现逻辑

奖励函数在 `drgrpo_grader.py` 中的 `r1_zero_reward_fn` 实现，它接受三个参数：

In [ ]:
def r1_zero_reward_fn(response: str, ground_truth: str, fast: bool = True) -> Dict[str, float]:
    # response: 模型生成的完整输出
    # ground_truth: 标准答案
    # fast: 是否使用快速评估模式
    # 返回包含 format_reward, answer_reward, reward 的字典
    if "</think> <answer>" in response and "</answer>" in response:
        model_answer = response.split("<answer>")[-1].replace("</answer>", "")
        if "\\boxed" in model_answer:
            model_answer = extract_answer(model_answer)
            if model_answer is None:
                return {
                    "format_reward": 1.0,
                    "answer_reward": 0.0,
                    "reward": 0.0
                }
        if isinstance(ground_truth, float) or isinstance(ground_truth, int):
            ground_truth = str(ground_truth)
        if isinstance(ground_truth, str):
            is_correct = grade(model_answer, ground_truth, fast)
        elif isinstance(ground_truth, list):
            is_correct = False
            for gt in ground_truth:
                is_correct |= grade(model_answer, gt, fast)
        if is_correct:
            return {
                "format_reward": 1.0,
                "answer_reward": 1.0,
                "reward": 1.0
            }
        else:
            # Formatted but wrong answer; no format reward to avoid hacking.
            return {
                "format_reward": 1.0,
                "answer_reward": 0.0,
                "reward": 0.0
            }
    else:
        # Unformatted.
        return {
            "format_reward": 0.0,
            "answer_reward": 0.0,
            "reward": 0.0
        }

**该函数实现的评估流程**：
1. **格式检查**：验证输出是否包含必需的标签
2. **答案提取**：从 `<answer>` 标签中解析最终答案
3. **答案比较**：使用数学等价性检查答案正确性
4. **奖励计算**：根据格式和答案正确性计算奖励分数

这个函数的设计使得评估过程既严格又灵活，能够准确衡量模型的推理能力和格式遵循能力。我们通过几个例子，测试下我们的奖励函数是够能正常工作！

In [8]:
# 导入奖励函数
try:
    from cs336_alignment.drgrpo_grader import r1_zero_reward_fn
    print("成功导入奖励函数")
except ImportError as e:
    print(f"导入奖励函数失败: {e}")
    print("请确保 drgrpo_grader.py 文件存在并且可以被导入")
    r1_zero_reward_fn = None

# 测试奖励函数的不同场景
def test_reward_function():
    if r1_zero_reward_fn is None:
        print("无法测试奖励函数，因为导入失败")
        return
    
    # 测试场景1：正确格式和答案
    print("\n=== 测试场景1：正确格式和答案 ===")
    correct_response = """</think>这是一个简单的加法问题。小明原来有5个苹果，又买了3个。所以总数是5 + 3 = 8个苹果。</think> <answer>8</answer>"""
    
    rewards = r1_zero_reward_fn(correct_response, "8")
    print("响应内容：")
    print(correct_response)
    print("\n奖励结果：")
    for key, value in rewards.items():
        print(f"  {key}: {value}")
    
    # 测试场景2：格式正确但答案错误
    print("\n=== 测试场景2：格式正确但答案错误 ===")
    wrong_answer_response = """<think>我来计算一下。5个苹果加3个应该是8个。等等，我算错了，是7个。</think> <answer>7</answer>"""
    
    rewards = r1_zero_reward_fn(wrong_answer_response, "8")
    print("奖励结果：")
    for key, value in rewards.items():
        print(f"  {key}: {value}")
    
    # 测试场景3：格式错误
    print("\n=== 测试场景3：格式错误（缺少标签） ===")
    bad_format_response = "这是一个简单的加法问题。小明原来有5个苹果，又买了3个。所以总数是5 + 3 = 8个苹果。答案是8。"
    
    rewards = r1_zero_reward_fn(bad_format_response, "8")
    print("响应内容：")
    print(bad_format_response)
    print("\n奖励结果：")
    for key, value in rewards.items():
        print(f"  {key}: {value}")

# 运行测试
test_reward_function()

成功导入奖励函数

=== 测试场景1：正确格式和答案 ===
响应内容：
</think>这是一个简单的加法问题。小明原来有5个苹果，又买了3个。所以总数是5 + 3 = 8个苹果。</think> <answer>8</answer>

奖励结果：
  format_reward: 1.0
  answer_reward: 1.0
  reward: 1.0

=== 测试场景2：格式正确但答案错误 ===
奖励结果：
  format_reward: 1.0
  answer_reward: 0.0
  reward: 0.0

=== 测试场景3：格式错误（缺少标签） ===
响应内容：
这是一个简单的加法问题。小明原来有5个苹果，又买了3个。所以总数是5 + 3 = 8个苹果。答案是8。

奖励结果：
  format_reward: 0.0
  answer_reward: 0.0
  reward: 0.0


## 1.4 使用 vLLM 进行高效推理

vLLM 是一个高性能的 LLM 推理引擎，具有以下优势：

1. **批量推理**：支持同时处理多个请求，提高吞吐量
2. **内存优化**：使用 PagedAttention 等技术优化显存使用
3. **CUDA 内核优化**：使用优化的 CUDA 内核加速推理
4. **灵活的采样参数**：支持温度采样、top-p 截断等多种采样策略

### vLLM 初始化参数详解

在 `evaluate_math.py` 中，我们这样初始化 vLLM：

In [ ]:
llm = LLM(
    model=MODEL_PATH,
    dtype=torch.bfloat16,  # 使用 bfloat16 精度节省显存
    gpu_memory_utilization=0.8,  # 控制显存使用率
    enable_prefix_caching=True,  # 启用前缀缓存优化
    tensor_parallel_size=1,  # 单 GPU 设置
)

### 采样参数配置

我们使用以下采样参数：
- `temperature=1.0`：使用随机采样，增加输出多样性，有助于探索不同的推理路径
- `top_p=1.0`：不进行 top-p 截断，使用完整的词汇分布
- `max_tokens=1024`：最大生成长度，足够容纳详细的推理过程
- `stop=["</answer>"]`：在生成答案标签时停止，确保输出格式完整
- `include_stop_str_in_output=True`：将停止字符串包含在输出中，便于后续解析

In [ ]:
# 初始化 vLLM 模型
import torch
from vllm import LLM, SamplingParams
from unittest.mock import patch
from vllm.model_executor import set_random_seed as vllm_set_random_seed

# 模型路径
MODEL_PATH = "/home/magnus-share/xuhu/model/Qwen2___5-Math-1___5B"

def init_vllm(model_path: str, device: str = "cuda:0"):
    """初始化 vLLM 模型
    
    初始化逻辑详解：
    1. 设置随机种子确保结果可重现
    2. 使用 patch 解决分布式训练相关问题
    3. 配置模型参数以优化性能和显存使用
    
    为什么需要 patch？
    - vLLM 默认假设在分布式环境中运行
    - 在单 GPU 环境中，我们需要模拟 world_size=1
    - profiling_patch 避免了不必要的性能分析
    """
    try:
        
        vllm_set_random_seed(42)
        
        world_size_patch = patch("torch.distributed.get_world_size", return_value=1)
        profiling_patch = patch(
            "vllm.worker.worker.Worker._assert_memory_footprint_increased_during_profiling",
            return_value=None
        )
        
        with world_size_patch, profiling_patch:
            llm = LLM(
                model=model_path,
                device=device,
                dtype=torch.bfloat16,  # 使用 bfloat16 节省显存
                enable_prefix_caching=True,  # 启用前缀缓存，提高批量推理效率
                gpu_memory_utilization=0.8,  # 控制显存使用率，避免 OOM
                tensor_parallel_size=1,  # 单 GPU 设置
                enforce_eager=True,  # 强制使用 eager 模式，提高稳定性
            )
            print(f"成功加载模型: {model_path}")
            return llm
            
    except Exception as e:
        print(f"初始化 vLLM 失败: {e}")
        print("请检查模型路径是否存在，以及是否有足够的显存")
        return None

# 初始化模型
print("正在初始化 vLLM 模型...")
llm = init_vllm(MODEL_PATH)

if llm is not None:
    print("vLLM 模型初始化完成！")
    
    # 设置采样参数
    sampling_params = SamplingParams(
        temperature=1.0,  # 随机采样，增加推理多样性
        top_p=1.0,        # 不进行 top-p 截断
        max_tokens=1024,  # 最大生成长度
        stop=["</answer>"],  # 遇到答案结束标签时停止
        include_stop_str_in_output=True,  # 包含停止字符串在输出中
    )
    
else:
    sampling_params = None
    print("vLLM 初始化失败，无法继续演示")

## 1.5 数据加载和预处理

数据加载是评估流程中的关键步骤。在 `evaluate_math.py` 中，我们实现了灵活的数据加载逻辑，支持多种数据集格式。

### 支持的数据集格式

1. **GSM8K 格式 (JSONL)**：
   - 每行一个 JSON 对象
   - 包含 `question` 字段和 `answer` 字段
   - 答案字段包含完整推理过程，最后以 `####` 分隔最终答案

2. **MATH 格式 (JSON 数组)**：
   - 整个文件是一个 JSON 数组
   - 包含 `problem` 和 `expected_answer` 字段
   - 答案是直接给出的最终结果

### 数据预处理逻辑

在 `evaluate_vllm` 函数中，我们实现了以下预处理步骤：

1. **格式检测**：通过读取第一个字符判断是 JSONL (`{`) 还是 JSON 数组 (`[`)
2. **字段提取**：根据数据集类型提取问题和答案
3. **答案解析**：对于 GSM8K，使用正则表达式提取 `####` 后的最终答案
4. **提示格式化**：将问题插入到 R1-Zero 提示模板中

### 答案提取策略

GSM8K 数据集的答案格式特殊，使用如下形式：
```
详细推理过程... #### 最终答案
```

我们使用正则表达式来提取最终答案，确保只获取正确答案部分。

In [9]:
# 数据加载函数 - 基于 evaluate_math.py 的实现
import json
import re

def load_math_dataset(dataset_path: str, max_samples: int = None):
    """
    加载数学数据集，支持 JSONL 和 JSON 数组两种格式
    
    这个函数的实现逻辑：
    1. 检测文件格式（JSONL vs JSON数组）
    2. 加载数据到内存
    3. 提取问题和答案字段
    4. 处理不同数据集格式的答案提取
    5. 格式化提示
    
    Args:
        dataset_path: 数据集文件路径
        max_samples: 最大加载样本数（用于演示）
        
    Returns:
        questions: 问题列表
        ground_truths: 标准答案列表
        prompts: 格式化后的提示列表
    """
    questions = []
    ground_truths = []
    
    try:
        # 步骤1：检测文件格式
        with open(dataset_path, 'r', encoding='utf-8') as f:
            first_char = f.read(1)
            f.seek(0)  # 重置文件指针
            
            if first_char == '[':
                # JSON 数组格式 (MATH数据集)
                print("检测到 JSON 数组格式 (MATH风格)")
                data = json.load(f)
            else:
                # JSONL 格式 (GSM8K数据集)
                print("检测到 JSONL 格式 (GSM8K风格)")
                data = []
                for line in f:
                    line = line.strip()
                    if line:  # 跳过空行
                        data.append(json.loads(line))
        
        print(f"成功加载了 {len(data)} 个样本")
        
        # 步骤2：限制样本数量（演示用）
        if max_samples:
            data = data[:max_samples]
            print(f"限制为前 {max_samples} 个样本用于演示")
        
        # 步骤3：处理每个样本
        for i, example in enumerate(data):
            # 提取问题 - 支持多种字段名
            question = example.get('question', example.get('problem', ''))
            
            # 提取答案 - 处理不同格式
            if 'expected_answer' in example:
                # MATH 格式：直接答案
                answer = example['expected_answer']
            elif 'answer' in example:
                # GSM8K 格式：从推理过程中提取最终答案
                answer_text = example['answer']
                # 使用正则表达式提取 #### 后的答案
                match = re.search(r"####\s*(.+)\s*$", answer_text.strip())
                answer = match.group(1).strip() if match else answer_text.strip()
            else:
                answer = ''
            
            questions.append(question)
            ground_truths.append(answer)
            
            # 只显示前几个样本的详细信息
            if i < 3:
                print(f"\n样本 {i+1}:")
                print(f"问题: {question}")
                print(f"原始答案字段: {example.get('answer', example.get('expected_answer', 'N/A'))}")
                print(f"提取的答案: {answer}")
        
        # 步骤4：格式化提示
        if 'r1_zero_template' in globals():
            prompts = [format_prompt(q, r1_zero_template) for q in questions]
            print(f"\n成功格式化了 {len(prompts)} 个提示")
        else:
            prompts = []
            print("\n提示模板未加载，无法格式化提示")
        
        return questions, ground_truths, prompts
        
    except FileNotFoundError:
        print(f"数据集文件未找到: {dataset_path}")
        return [], [], []
    except Exception as e:
        print(f"加载数据集时出错: {e}")
        return [], [], []

# 加载数据（演示用小样本）
DATASET_PATH = "data/gsm8k/test.jsonl"
questions, ground_truths, prompts = load_math_dataset(DATASET_PATH, max_samples=10)

print(f"\n总共加载了 {len(questions)} 个问题")
if prompts:
    print(f"示例提示预览：")
    print("-" * 50)
    print(prompts[0][:200] + "..." if len(prompts[0]) > 200 else prompts[0])
    print("-" * 50)

检测到 JSONL 格式 (GSM8K风格)
成功加载了 1319 个样本
限制为前 10 个样本用于演示

样本 1:
问题: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
原始答案字段: Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18
提取的答案: 18

样本 2:
问题: A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?
原始答案字段: It takes 2/2=<<2/2=1>>1 bolt of white fiber
So the total amount of fabric is 2+1=<<2+1=3>>3 bolts of fabric
#### 3
提取的答案: 3

样本 3:
问题: Josh decides to try flipping a house.  He buys a house for $80,000 and then puts in $50,000 in repairs.  This increased the value of the house by 150%.  How much profit did he make?
原始答案字段: The cost of the house and repairs came out to 80,000+50,

## 1.6 评估指标计算

现在我们可以使用 vLLM 对所有问题进行批量推理，并使用奖励函数评估模型的性能。

In [ ]:
fr = int(rewards.get("format_reward", 0.0) >= 0.5)
ar = int(rewards.get("answer_reward", 0.0) >= 0.5)
combo_counts[(fr, ar)] += 1

### 组合分析 (Combo Analysis)

我们统计四种组合的情况：
- `(1, 1)`：格式正确且答案正确
- `(1, 0)`：格式正确但答案错误
- `(0, 0)`：格式错误且答案错误
- `(0, 1)`：格式错误但答案正确（理论上很少见）

这种分析有助于我们理解模型在格式遵循和推理能力方面的表现。

In [ ]:
metrics = {
    "n": num_examples,
    "format_rate": avg_format_reward,
    "answer_accuracy": avg_answer_reward,
    "reward_mean": avg_reward,
    "counts": {
        "format=1 answer=1": combo_counts[(1, 1)],
        "format=1 answer=0": combo_counts[(1, 0)],
        "format=0 answer=0": combo_counts[(0, 0)],
        "format=0 answer=1": combo_counts[(0, 1)],
    }
}

## 1.7 结果分析

通过运行`evaluate_math.py`脚本，我们可以完成对`Qwen2.5-Math-1.5B`模型的完整评估。运行完成后，你会得到类似以下的输出：

```json
{
  "n": 1319,
  "format_rate": 0.5041698256254739,
  "answer_accuracy": 0.17589082638362397,
  "reward_mean": 0.17589082638362397,
  "counts": {
    "format=1 answer=1": 232,
    "format=1 answer=0": 433,
    "format=0 answer=0": 654,
    "format=0 answer=1": 0
  }
}
```

基准模型 `Qwen2.5-Math-1.5B` 的回答准确率是 `17.59%`，格式准确率为 `50.42%`。接着我们观察了 10 个格式奖励为 0 的案例，我们发现问题主要出在解析器的格式要求过于严格，而不是基础模型的输出质量问题。从 `r1_zero_reward_fn` 的代码看，格式检查的条件是：

```python
if "</think> <answer>" in response and "</answer>" in response:
```

这个条件要求：
- 响应中必须包含确切的字符串 `</think> <answer>` （注意 think 结束标签和 answer 开始标签之间必须有空格）
- 响应中必须包含 `</answer>`

而在多个案例中我们发现模型输出通常为 `</think>\n<answer>` 或 `</think><answer>`（没有空格），不符合解析器期望的 `</think> <answer>`（必须有空格）

对于格式正确但答案错误的案例，问题主要出在基础模型的推理能力上，而不是解析器的问题。

# 第二部分：用于数学推理的监督微调 (SFT)


### 学习目标
在本部分中，我们将学习如何通过监督微调来提升语言模型的数学推理能力。这包括：
1. 理解 SFT 在数学推理中的作用和原理
2. 掌握推理轨迹数据的准备和处理
3. 实现完整的 SFT 训练流程
4. 学习模型评估和性能分析技巧

## 2.1 SFT 的动机和原理

### 为什么需要 SFT？

尽管 Qwen 2.5 Math 1.5B 已经经过数学预训练，但在推理任务上仍然表现不佳：
- **缺乏推理结构**：模型不知道如何进行逐步推理
- **格式不一致**：不知道使用 `<think>` 和 `<answer>` 标签
- **推理模式缺失**：没有学习到思维链（Chain-of-Thought）推理模式，推理能力较弱、深度不够

### SFT 的核心思想

SFT通过让模型学习包含`数学问题+推理过程+最终答案`的推理轨迹，使其能够生成完整的推理过程，从而提供明确的推理模式和步骤以提升性能。

## 2.2 数据准备和预处理

### 推理轨迹数据

由于原项目的带推理轨迹的 MATH 12K 数据没有公开，我们使用来自`garg-aayush`用户复现该项目时制作的[hiyouga/math12k](https://huggingface.co/datasets/hiyouga/math12k) 数据集。该数据集使用 gpt-oss 蒸馏的[带推理轨迹的数据](https://huggingface.co/datasets/garg-aayush/sft-cs336-assign5-datasets/tree/main/sft-reason) 的推理轨迹数据：

```json
# 数据格式示例
{
    "problem": "Compute $\\sin 45^\\circ$.",
    "reasoning_trace": "The angle 45\u00b0 corresponds to \u03c0/4 radians. The sine of \u03c0/4 is known from the unit circle or a 45-45-90 right triangle, where the legs are equal and the hypotenuse is \u221a2 times a leg. Thus, sin\u202f45\u00b0 = opposite/hypotenuse = 1/\u221a2 = \u221a2/2.</think> <answer>\u221a2/2</answer>",
    "extracted_answer": "\u221a2/2",
    "expected_answer": "\\frac{\\sqrt{2}}{2}"
}
```

### 数据加载逻辑

In [ ]:
# 处理不同格式的数据文件
with open(train_data_path, 'r', encoding='utf-8') as f:
    first_char = f.read(1)
    f.seek(0)

    if first_char == '[':
        # JSON 数组格式
        sft_data = json.load(f)
    else:
        # JSONL 格式
        sft_data = [json.loads(line) for line in f if line.strip()]

# 数据子采样（用于不同大小实验）
if max_examples > 0 and len(sft_data) > max_examples:
    sft_data = random.sample(sft_data, max_examples)

## 2.3 SFT 训练的核心组件

下面的所有辅助组件均定义在`sft_helper.py`，可以在该脚本中查看完整实现。

### 1. 提示和输出的分词 (`tokenize_prompt_and_output`)

这是 SFT 的基础函数，用于准备训练数据：

In [ ]:
def tokenize_prompt_and_output(
        prompt_strs: List[str],
        output_strs: List[str],
        tokenizer: PreTrainedTokenizerBase,
) -> Dict[str, torch.Tensor]:

1. **分别对提示和输出进行分词**

In [ ]:
prompt_tok = tokenizer(prompt_strs, add_special_tokens=False, padding=False)
output_tok = tokenizer(output_strs, add_special_tokens=False, padding=False)

2. **拼接完整序列**

In [ ]:
full_ids_list = [p + o for p, o in zip(prompt_ids_list, output_ids_list)]

3. **填充到相同长度**


In [ ]:
max_full_len = max(len(x) for x in full_ids_list)
full_padded = torch.full((bs, max_full_len), pad_id, dtype=torch.long)

4. **创建训练标签**

In [ ]:
input_ids = full_padded[:, :-1]  # 去掉最后一个token
labels = full_padded[:, 1:]      # 去掉第一个token

5. **构建响应掩码**


In [ ]:
response_mask = torch.zeros_like(labels, dtype=torch.long)
for i in range(bs):
    p_len = prompt_lens[i]
    o_len = output_lens[i]
    start = max(p_len - 1, 0)
    end = min(p_len + o_len - 1, labels.size(1))
    response_mask[i, start:end] = 1


**返回结构：**
```python
{
    "input_ids": input_ids,        # (B, T) - 模型输入
    "labels": labels,             # (B, T) - 目标标签（右移一位）
    "response_mask": response_mask # (B, T) - 响应部分掩码
}
```

### 2. 对数概率计算 (`get_response_log_probs`)

计算模型对目标序列的条件对数概率：

In [ ]:
def get_response_log_probs(
        model: torch.nn.Module,
        input_ids: torch.Tensor,
        labels: torch.Tensor,
        return_token_entropy: bool = False,
) -> Dict[str, torch.Tensor]:

**计算过程：**
1. **前向传播**：`logits = model(input_ids).logits`
2. **转换为对数概率**：`log_probs_vocab = F.log_softmax(logits, dim=-1)`
3. **选择目标token的概率**：


In [ ]:
import torch.nn.functional as F

# Forward: logits (B, T, V)
logits = model(input_ids).logits

# log-probs over vocab: (B, T, V)
log_probs_vocab = F.log_softmax(logits, dim=-1)

# Select log-prob of the label token at each position.
# labels: (B, T) -> (B, T, 1) for gather
log_probs = torch.gather(log_probs_vocab, dim=-1, index=labels.unsqueeze(-1)).squeeze(-1)

### 3. 熵计算 (`compute_entropy`)

用于监控模型预测的确定性。**高熵**意味着模型对下一个token不确定，预测分布平坦；**低熵**表示模型很确定下一个token，预测分布集中。

In [ ]:
def compute_entropy(logits: torch.Tensor) -> torch.Tensor:
    # 计算每个位置的预测熵
    log_z = torch.logsumexp(logits, dim=-1)  # log Z
    probs = torch.softmax(logits, dim=-1)
    expected_logit = (probs * logits).sum(dim=-1)
    entropy = log_z - expected_logit  # H = log Z - E_p[logit]
    return entropy

### 4. 掩码归一化 (`masked_normalize`)

用于在响应token上进行归一化的工具函数：

In [ ]:
def masked_normalize(
        tensor: torch.Tensor,
        mask: torch.Tensor,
        normalize_constant: float,
        dim: int | None = None,
) -> torch.Tensor:
    masked_tensor = tensor * mask
    summed = masked_tensor.sum() if dim is None else masked_tensor.sum(dim=dim)
    return summed / normalize_constant

### 5. 微批次训练步骤 (`sft_microbatch_train_step`)

实现单个训练步骤的核心逻辑：

In [ ]:
def sft_microbatch_train_step(
        policy_log_probs: torch.Tensor,
        response_mask: torch.Tensor,
        gradient_accumulation_steps: int,
        normalize_constant: float = 1.0,
) -> Tuple[torch.Tensor, Dict[str, torch.Tensor]]:

**训练步骤：**
1. **计算负对数似然**：`per_token_nll = -policy_log_probs`
2. **掩码求和**：`per_example_nll = (per_token_nll * mask).sum(dim=1)`
3. **归一化**：`per_example_nll = per_example_nll / normalize_constant`
4. **批次平均**：`microbatch_loss = per_example_nll.mean()`
5. **梯度累积缩放**：`loss = microbatch_loss / gradient_accumulation_steps`
6. **反向传播**：`loss.backward()`

## 2.4 训练流程和超参数

训练流程定义在`run_sft_experiment`函数中，该函数会在主函数中被调用，并通过传递不同参数进行不同的实验：

- `train_data_path`: 训练数据所在路径
- `max_examples`: 取值为`-1`表示使用所有数据，也可以采样 128, 256, 512, 1024, 2048 等子集
- `dataset_tag`: 取值为`raw`和`filtered`，表示使用原数据还是过滤后的数据
- `size_tag`: 输入为一个字符串，用来标记使用的样本大小，可以为"full"、"1024"等

In [ ]:
def run_sft_experiment(train_data_path: str, max_examples: int = -1,
                      dataset_tag: str = "raw", size_tag: str = "full"):
    """Run a single SFT experiment."""

### 初始化 wandb

wandb 是一个用来记录深度学习实验的工具，如果你是第一次使用它，请先参考我们的教程`docs/chapter0/Weights & Biases（W&B）使用介绍.md`。

### 加载提示模版

In [ ]:
PROMPT_TEMPLATE_PATH = "cs336_alignment/prompts/r1_zero.prompt"

template = load_prompt_template(PROMPT_TEMPLATE_PATH)

### 加载训练数据

In [ ]:
print(f"Loading training data from {train_data_path}...")
sft_data = []
with open(train_data_path, 'r', encoding='utf-8') as f:
    # Check first char to see if it's a list '[' or a dict '{'
    first_char = f.read(1)
    f.seek(0)

    if first_char == '[':
        # Case A: It is a JSON List (Your format)
        print("Detected JSON List format.")
        sft_data = json.load(f)
    else:
        # Case B: It is JSONL (Line-by-line)
        print("Detected JSONL format.")
        for line in f:
            line = line.strip()
            if not line: continue
            try:
                sft_data.append(json.loads(line))
            except json.JSONDecodeError:
                continue

# Subsample if needed
if max_examples > 0 and len(sft_data) > max_examples:
    sft_data = random.sample(sft_data, max_examples)
    print(f"Subsampled training data to {len(sft_data)} examples.")

### 初始化策略模型


In [ ]:
print("Initializing Policy Model on cuda:0...")
device_policy = "cuda:0"
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2"
).to(device_policy)

optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

### 初始化 vLLM

In [ ]:
print("Initializing vLLM on cuda:1...")
device_eval = "cuda:1"
llm = init_vllm(BASE_MODEL, device_eval, SEED)

# Evaluation sampling parameters
eval_sampling_params = SamplingParams(
    temperature=0.0, top_p=1.0, max_tokens=1024,
    stop=["</answer>"], include_stop_str_in_output=True
)

### 训练超参数

In [ ]:
BATCH_SIZE = 4          # 微批次大小
GRAD_ACCUM = 8          # 梯度累积步数
LR = 5e-5              # 学习率
EPOCHS = 1             # 训练轮数
MAX_GRAD_NORM = 1.0    # 梯度裁剪
SEED = 2026   # 随机种子

### 训练循环

In [ ]:
global_step = 0
model.train()

# Calculate eval frequency (roughly every half epoch)
steps_per_epoch = max(1, len(sft_data) // BATCH_SIZE)
eval_every_steps = EVAL_EVERY_STEPS
print(f"Training on {len(sft_data)} examples, {steps_per_epoch} steps per epoch, eval every {eval_every_steps} steps")

eval_count = 0
batch_loss = 0.0

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch + 1}/{EPOCHS}")
    random.shuffle(sft_data)

    for i in tqdm(range(0, len(sft_data), BATCH_SIZE), desc=f"Epoch {epoch + 1}"):
        # 1. 准备批次数据
        batch_data = sft_data[i : i + BATCH_SIZE]
        if not batch_data:
            continue

        # 2. 格式化提示和响应
        prompt_strs = []
        output_strs = []

        for x in batch_data:
            question = x.get('problem', x.get('question', ''))
            formatted_prompt = format_prompt(template, question)

            # Handle different response keys
            if 'reasoning_trace' in x:
                response = x['reasoning_trace']
            elif 'response' in x:
                response = x['response']
            else:
                continue

            prompt_strs.append(formatted_prompt)
            output_strs.append(response)

        if not prompt_strs:
            continue

        # 3. 分词
        tokenized = tokenize_prompt_and_output(prompt_strs, output_strs, tokenizer)
        input_ids = tokenized["input_ids"].to(device_policy)
        labels = tokenized["labels"].to(device_policy)
        response_mask = tokenized["response_mask"].to(device_policy)
        
        # 4. 前向传播
        outputs = model(input_ids)
        logits = outputs.logits

        log_probs = torch.log_softmax(logits, dim=-1)
        per_token_log_probs = torch.gather(
            log_probs,
            dim=2,
            index=labels.unsqueeze(2)
        ).squeeze(2)

        # 5. 计算损失并反向传播
        loss, _ = sft_microbatch_train_step(
            policy_log_probs=per_token_log_probs,
            response_mask=response_mask,
            gradient_accumulation_steps=GRAD_ACCUM
        )
        batch_loss += loss.item()

        # 6. 梯度累积和优化器步骤
        if (global_step + 1) % GRAD_ACCUM == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)
            optimizer.step()
            optimizer.zero_grad()

            # Log training metrics
            wandb.log({
                "train/loss": batch_loss,
                "train_step": global_step
            })
            batch_loss = 0.0

        # 7. 定期评估
        if (global_step + 1) % eval_every_steps == 0:
            print(f"Step {global_step}: Evaluating...")
            model.eval()
            load_policy_into_vllm_instance(model, llm)

            eval_dir = f"results/sft_experiments_{dataset_tag}_{size_tag}/step_{global_step}"
            os.makedirs(eval_dir, exist_ok=True)

            # Use GSM8K test set for evaluation
            metrics = evaluate_vllm(
                vllm_model=llm,
                reward_fn=r1_zero_reward_fn,
                dataset_path=VAL_PATH,
                prompt_template=template,
                eval_sampling_params=eval_sampling_params,
                output_filepath=os.path.join(eval_dir, "results.jsonl")
            )

            wandb.log({
                "eval/acc": metrics["answer_accuracy"],
                "eval/format_rate": metrics["format_rate"],
                "eval_step": eval_count,
                "global_step": global_step
            })
            eval_count += 1

            print(f"Eval Acc: {metrics['answer_accuracy']:.4f}")

            # Log generations to wandb
            log_generations_to_wandb(
                results_path=os.path.join(eval_dir, "results.jsonl"),
                step=global_step,
                run_name=f"sft_{dataset_tag}_{size_tag}"
            )

            model.train()

        global_step += 1

# Save final model（是否保存最后的模型，取消注释即可保存模型）
# print("Saving final model...")
# final_save_dir = os.path.join(f"results/sft_experiments_{dataset_tag}_{size_tag}", "latest")
# model.save_pretrained(final_save_dir)
# tokenizer.save_pretrained(final_save_dir)
# print(f"Final model saved to {final_save_dir}")

print("Training Complete.")
wandb.finish()

## 2.5 实验设置

### 1. 数据集大小扫描实验

**目的：** 研究训练数据量对性能的影响，找到数据效率的平衡点。


In [ ]:
DATASET_SIZES = [128, 256, 512, 1024]

for size in DATASET_SIZES:
    run_sft_experiment(
        train_data_path=RAW_TRAIN,
        max_examples=size,
        dataset_tag="raw",
        size_tag=str(size)
    )

### 2. 过滤正确示例实验

**动机：**移除低质量的推理轨迹，提高训练数据的质量，减少模型学习错误推理模式的概率。

In [ ]:
# 预处理：只保留产生正确答案的推理轨迹
new_sft_data = []
for x in sft_data:
    score = r1_zero_reward_fn(x['reasoning_trace'], x['expected_answer'])
    if score['answer_reward'] == 1.0:
        new_sft_data.append(x)

### 3. 性能指标监控

**训练指标：**
- `train/loss`：训练损失
- `train/entropy`：响应token的平均熵

**评估指标：**
- `eval/acc`：答案准确率
- `eval/format_rate`：格式正确率

## 2.6 实验结果分析


### 数据集大小的影响

<center class="half">
    <img src="images/sft_train_loss.png" width="400"/>
    <img src="images/sft_eval_acc.png" width="400"/>
    <img src="images/sft_eval_format.png" width="400"/>
</center>

从图中可以清晰地看到，数据集越大，最终的训练损失越低。而在评测集上的精确度 512 条数据时表现最好，能达到 `76.20%`。我们的一篇研究 [Structure Trumps Size: Rethinking Data Quality for LLM Reasoning](https://aclanthology.org/2025.findings-emnlp.616/) 发现，大的趋势是数据越多越好，但是 1k 之前带来的收益最大，在超过 10k 后收益持续变小，并且会对模型的通用能力损失较大。

<center align="center">
    <img src="images/sft_volume.png" width="400"/>
</center>

### 过滤数据的效果

<center align="center">
    <img src="images/sft_256_raw_vs_filter.png" width="400"/>
</center>

移除错误推理轨迹通常能带来更好地性能提升，但是在此数据上不明显，在 256 条数据时，过滤后只是从 75.50% 提升到 76.20%。但是相较于基座模型提升很大（17.59% --> 76.20%）。我们进一步将`简单移除错误`这种过滤方法，扩展到提升`数据质量`这一概念上，我们猜想通过提升数据质量这种方法应该同样会有不错的性能，现有的研究如[s1]()、[LIMO]()等使用`1k`左右的高质量数据就可以得到很好的推理性能。但是需要注意的是他们的结果通常对基座模型的推理能力有要求，在`Qwen2.5-32B-Instruct`上有较为不错的提升，但是随着模型规模变小，单纯使用少量高质量数据的效果通常没那么好。

# 第三部分：专家迭代 (Expert Iteration)

### 学习目标

在本部分中，我们将学习专家迭代算法，这是一种结合了自我生成数据和监督学习的强大方法。这包括：

1. 理解专家迭代的原理和优势
2. 掌握迭代式推理能力提升的方法
3. 实现完整的专家迭代训练流程
4. 学习超参数调优和性能分析

## 3.1 专家迭代的原理

在前一节中，我们发现通过从监督微调（SFT）数据中剔除质量差的样本，可以提升 SFT 模型的性能。而在本节中，我们更进一步：不再只依赖外部提供的 SFT 数据，而是让模型自己（即“base model”，基础模型）生成推理轨迹（reasoning traces），然后对这些自生成的推理过程进行同样的“过滤”操作——只保留高质量的样本用于后续训练。这种“让模型自己生成数据 → 筛选高质量样本 → 用这些样本再训练自己”的迭代式自我提升方法，被称为 [Expert Iteration（专家迭代）](https://arxiv.org/pdf/1705.08439)。

在大语言模型（LLM）背景下应用该思想的代表性工作，例如 [Cobbeet al. [2021b]](https://arxiv.org/pdf/2110.14168), [Zelikman et al. [2022]](https://arxiv.org/pdf/2203.14465), [Dohan et al. [2022]](https://arxiv.org/pdf/2207.10342), [Gulcehre et al. [2023]](https://arxiv.org/pdf/2308.08998).

### 为什么专家迭代有效？

- **自举学习**：模型从自己的成功案例中学习，避免了人工标注的高成本
- **持续改进**：每次迭代都能生成更好的推理轨迹
- **数据效率**：专注于高质量的推理模式
- **可扩展性**：可以持续进行多轮迭代

### 与纯SFT的区别

| 方面 | 纯SFT | 专家迭代 |
|------|-------|----------|
| 数据来源 | 预先准备的推理轨迹 | 模型自我生成的轨迹 |
| 数据质量 | 依赖人工标注或强模型 | 通过正确性过滤保证质量 |
| 迭代性 | 一次性训练 | 多轮自举改进 |
| 适应性 | 固定数据 | 随模型改进而改进 |

## 3.2 算法流程详解

### 整体架构

专家迭代的核心循环如下：

In [ ]:
for step in range(N_EI_STEPS):
    # A. 使用当前模型生成大量推理轨迹
    rollouts = generate_rollouts(current_model, questions, G)

    # B. 过滤出正确的推理轨迹作为"专家"数据
    expert_data = filter_correct_rollouts(rollouts)

    # C. 在专家数据上进行监督学习
    current_model = supervised_finetuning(current_model, expert_data)

    # D. 评估性能提升
    evaluate_model(current_model)

### 关键超参数

- **G (rollouts_per_question)**：每个问题生成的推理轨迹数量
- **Db (expert_batch_size)**：每次迭代使用的训练问题数量
- **epochs_per_step**：每次迭代的SFT训练轮数
- **N_EI_STEPS**：总的迭代轮数

## 3.3 核心实现组件

### 推理轨迹生成

In [ ]:
# 采样参数设置
rollout_params = SamplingParams(
    temperature=1.0,      # 高温度鼓励多样性
    top_p=1.0,
    max_tokens=1024,
    min_tokens=4,         # 防止空响应
    stop=["</answer>"],
    include_stop_str_in_output=True,
    n=rollouts_per_question  # 每个问题生成G个轨迹
)

# 生成rollouts
outputs = llm.generate(prompts, rollout_params)

### 正确性过滤

过滤标准是只保留答案正确的推理轨迹。

In [ ]:
# 过滤逻辑
new_sft_data = []
correct_count = 0

for i, req_output in enumerate(outputs):
    gt = ground_truths[i]
    prompt = prompts[i]

    for completion in req_output.outputs:
        generated_text = completion.text

        # 使用奖励函数验证正确性
        score = r1_zero_reward_fn(generated_text, gt)

        if score['answer_reward'] == 1.0:
            correct_count += 1
            new_sft_data.append({
                "prompt": prompt,
                "response": generated_text
            })

### 熵计算监控

- **训练初期**：熵较高，表示模型不确定
- **训练中期**：熵逐渐降低，表示模型学会了推理模式
- **收敛状态**：熵稳定，表示模型达到最优

In [ ]:
def compute_mean_entropy(logits, mask):
    """
    计算响应token的平均熵，用于监控模型确定性
    """
    probs = F.softmax(logits, dim=-1)
    log_probs = F.log_softmax(logits, dim=-1)

    # 熵 = -sum(p * log p)
    entropy = -torch.sum(probs * log_probs, dim=-1)

    # 只计算响应token的熵
    masked_entropy = entropy * mask
    sum_mask = mask.sum()
    return masked_entropy.sum() / sum_mask if sum_mask > 0 else torch.tensor(0.0)

## 3.4 实验设置和超参数调优

### 主要实验变量

In [ ]:
# 数据集大小扫描
EXPERT_BATCH_SIZES = [512, 1024, 2048]  # Db

# Rollouts per question
ROLLOUTS_PER_QUESTION = [1, 4, 8, 16]   # G

# SFT epochs per step
SFT_EPOCHS_PER_STEP = [4, 8, 16]     # epochs

### 实验配置组合

主要实验设置：

1. **固定G=4, epochs=4，变化Db**：研究数据集大小的影响
2. **固定Db=512, epochs=4，变化G**：研究rollouts数量的影响
3. **固定Db=512, G=4，变化epochs**：研究SFT训练深度

### 训练流程

In [ ]:
for step in range(N_EI_STEPS):
    print(f"=== Expert Iteration Step {step+1}/{N_EI_STEPS} ===")

    # 1. 同步权重到vLLM
    load_policy_into_vllm(policy, llm)

    # 2. 生成rollouts
    print(f"Generating {len(prompts) * rollouts_per_question} rollouts...")
    outputs = llm.generate(prompts, rollout_params)

    # 3. 过滤正确示例
    new_sft_data = filter_correct_examples(outputs, prompts, ground_truths)

    # 4. SFT训练
    print(f"Training on {len(new_sft_data)} examples...")
    policy = sft_train_step(policy, new_sft_data, sft_epochs_per_step)

    # 5. 评估
    evaluate_performance(policy, llm, val_data)

## 3.5 性能分析和结果解读


### 关键指标监控


**训练指标：**
- `ei/correct_rate`：每轮rollouts的正确率
- `ei/dataset_size`：专家数据集大小
- `train/loss`：SFT训练损失
- `train/entropy`：响应token熵

**评估指标：**
- `eval/acc`：验证集准确率
- `eval/format_rate`：格式正确率

### 实验结果分析

#### 1. 数据集大小 (Db) 的影响

<center class="half">
    <img src="images/sft_ei_Db_train_loss.png" width="300"/>
    <img src="images/sft_ei_Db_train_entropy.png" width="300"/>
    <img src="images/sft_ei_Db_eval_acc.png" width="300"/>
</center>

- Db = 512:  accuracy = 63.08%
- Db = 1024: accuracy = 67.17%
- Db = 2048: accuracy = 66.49%

在此实验中，Db=1024 表现出最佳的最终准确率 (67.17%)，而 Db=512 在训练初期表现最好，Db=2048 虽然最终准确率略低于 Db=1024，但其训练过程最为稳定，最终的熵值更低。

#### 2. Rollouts数量 (G) 的影响

<center class="half">
    <img src="images/sft_ei_rollout_train_loss.png" width="300"/>
    <img src="images/sft_ei_rollout_train_entropy.png" width="300"/>
    <img src="images/sft_ei_rollout_eval_acc.png" width="300"/>
</center>

- G = 1:  accuracy = 66.49%
- G = 4:  accuracy = 67.17%
- G = 8:  accuracy = 66.19%
- G = 16: accuracy = 65.96%

在此实验中，G=4 表现出最佳的最终准确率 (67.17%)。随着 G 值的增加，模型的最终性能呈现先升后降的趋势。这表明存在一个“最优”的采样多样性水平：过少的采样会导致数据不足，而过多的采样则会引入噪声和低质量样本，反而损害模型性能。

#### 3. 训练深度 (epochs) 的影响

<center class="half">
    <img src="images/sft_ei_epoch_train_loss.png" width="300"/>
    <img src="images/sft_ei_epoch_train_entropy.png" width="300"/>
    <img src="images/sft_ei_epoch_eval_acc.png" width="300"/>
</center>


- epochs = 4:  optimal = 63.08%
- epochs = 8:  good_fit = 63.46%
- epochs = 16: overfitting = 71.80%

在此实验中，epochs=16 表现出最高的最终准确率 (71.80%)，显著优于 epochs=4 (63.08%) 和 epochs=8 (63.46%)。这表明，在专家迭代框架下，对高质量的"专家"数据进行更深度的训练是提升模型性能的关键。尽管 epochs=16 的曲线在中期出现波动，但其最终收敛效果最好，证明了其强大的学习能力。


# 第四部分：分组相对策略优化 (GRPO)

### 学习目标

在本部分中，我们将学习分组相对策略优化 (Group Relative Policy Optimization, GRPO)，这是一种专为语言模型推理任务设计的强化学习算法。学习内容包括：

1. 理解 GRPO 的核心原理和相对于传统 RL 的优势
2. 掌握分组归一化优势的计算方法
3. 实现多种策略梯度损失函数
4. 构建完整的 GRPO 训练循环

## 4.1 GRPO 的核心原理


### 什么是 GRPO？

`GRPO (Group Relative Policy Optimization)`是 Deepseek 在 [DeepSeekMath](https://arxiv.org/abs/2402.03300) 论文里首次提出的强化学习算法，旨在解决传统 `PPO（Proximal Policy Optimization）`在大规模模型训练中存在的计算效率低和内存占用高的问题。GRPO 和专家迭代一样，也是一种在线学习算法（online learning algorithm），这意味着它通过使用训练过程中由训练模型自身生成的数据来迭代改进。

<center align="center">
    <img src="../../docs/chapter14/images/14-4-ppo与grpo的对比.png" width="800"/>
</center>


标准的 PPO 通常需要加载四个模型：
- `Policy Model (Actor)`：当前训练的策略模型。
- `Reference Model`：用于计算 KL 散度，防止模型跑偏。
- `Reward Model`：用于给输出打分。
- `Value Model (Critic)`：用于估计状态价值 $V(s)$，以计算优势函数。

这使得训练所需的显存几乎是推理时的 3-4 倍。`价值模型`本身通常也需要是一个巨大的神经网络，训练它需要额外的计算资源。

GRPO 的核心思想是：不再训练一个额外的价值模型来评估当前状态的好坏，而是通过对同一个提示（Prompt）采样多组输出，计算这些输出在组内的相对优势（Relative Advantage），即比较这些输出的“相对好坏”。


## 4.2 分组归一化优势计算


### 核心思想

GRPO 的核心创新是**分组相对优势**的计算：

对于一个问题 $q$，从策略 $π_θ$ 中采样 $G$ 个响应 ${o^{i}}_{i=1}^G$，计算每个响应的奖励 $r^{i} = R(q, o^{i})$，然后计算分组归一化的优势：

$$
A^{i} = (r^{i} - mean(r^{1}, r^{2}, ..., r^{G})) / (std(r^{1}, r^{2}, ..., r^{G}) + advantage_{eps})
$$

这个优势 $A^{i}$ 对于响应中的每个 token 都是相同的。

### 实现细节

In [ ]:
def compute_group_normalized_rewards(
        reward_fn: Callable[[str, str], Dict[str, float]],
        rollout_responses: List[str],
        repeated_ground_truths: List[str],
        group_size: int,
        advantage_eps: float,
        normalize_by_std: bool,
) -> Tuple[torch.Tensor, torch.Tensor, Dict[str, float]]:

**关键实现步骤：**

1. **计算原始奖励**

In [ ]:
raw_rewards_list: List[float] = []
raw_format_list: List[float] = []
raw_answer_list: List[float] = []

for resp, gt in zip(rollout_responses, repeated_ground_truths):
    scores = reward_fn(resp, gt)  # expected keys: reward/format_reward/answer_reward
    # Be strict so bugs surface early
    if "reward" not in scores:
        raise KeyError(f"reward_fn output missing key 'reward': {scores.keys()}")
    raw_rewards_list.append(float(scores["reward"]))
    raw_format_list.append(float(scores.get("format_reward", 0.0)))
    raw_answer_list.append(float(scores.get("answer_reward", 0.0)))

raw_rewards = torch.tensor(raw_rewards_list, dtype=torch.float32)
raw_format = torch.tensor(raw_format_list, dtype=torch.float32)
raw_answer = torch.tensor(raw_answer_list, dtype=torch.float32)

2. **计算组归一化**

In [ ]:
# 分组重组
rollout_batch_size = len(rollout_responses)
n_groups = rollout_batch_size // group_size
rewards_g = raw_rewards.view(n_groups, group_size)  # (n_groups, group_size)

group_mean = rewards_g.mean(dim=1, keepdim=True)  # (n_groups, 1)
centered = rewards_g - group_mean  # (n_groups, group_size)

if normalize_by_std:
    # std per group; use unbiased=False for stability (population std)
    group_std = rewards_g.std(dim=1, keepdim=True, unbiased=True)  # (n_groups, 1)
    denom = group_std + float(advantage_eps)
    advantages_g = centered / denom
    zero_std_groups = (group_std.squeeze(1) == 0).sum().item()
else:
    advantages_g = centered
    group_std = rewards_g.std(dim=1, keepdim=True, unbiased=True)
    zero_std_groups = (group_std.squeeze(1) == 0).sum().item()

advantages = advantages_g.reshape(-1)  # (rollout_batch_size,)

### 优势的含义

- **正优势 (A > 0)**：该响应在组内表现优于平均水平，应该增加其概率
- **负优势 (A < 0)**：该响应在组内表现劣于平均水平，应该减少其概率
- **零优势 (A = 0)**：该响应在组内表现等于平均水平，不改变其概率

### 元数据收集

函数还返回详细的统计信息用于调试和监控：

In [ ]:
metadata = {
    "rollout_batch_size": float(rollout_batch_size),
    "n_groups": float(n_groups),
    "group_size": float(group_size),
    "raw_reward_mean": float(raw_rewards.mean().item()),
    "raw_reward_std": float(raw_rewards.std(unbiased=False).item()),
    "group_reward_mean_mean": float(group_means.mean().item()),
    "adv_mean": float(advantages.mean().item()),
    "adv_std": float(advantages.std(unbiased=False).item()) if advantages.numel() > 1 else 0.0,
    # ... 更多统计
}

## 4.3 策略梯度损失函数

### 朴素策略梯度损失

最基本的策略梯度损失，**计算公式** 为 $loss_{b,t} = -A_{b} * log π_θ(o_{b,t} | q_b, o_{b,<t})$


In [ ]:
def compute_naive_policy_gradient_loss(
        raw_rewards_or_advantages: torch.Tensor,  # (batch_size, 1)
        policy_log_probs: torch.Tensor,          # (batch_size, sequence_length)
) -> torch.Tensor: 

**实现：**

In [ ]:
advantages = raw_rewards_or_advantages.to(dtype=policy_log_probs.dtype, device=policy_log_probs.device)
loss = -(advantages * policy_log_probs)  # 广播: (B,1) -> (B,T)
return loss

### GRPO-Clip 损失

带裁剪机制的策略梯度损失，**核心公式**为：

$$
ratio_{b,t} = π_θ(o_{b,t}) / π_{θ_{old}}(o_{b,t})  
$$

$$
clipped_{ratio} = clip(ratio, 1-ε, 1+ε)  
$$

$$
loss_{b,t} = -min(ratio_{b,t} * A_b, clipped_ratio_{b,t} * A_b)  
$$

In [ ]:
def compute_grpo_clip_loss(
        advantages: torch.Tensor,           # (batch_size, 1)
        policy_log_probs: torch.Tensor,     # (batch_size, sequence_length)
        old_log_probs: torch.Tensor,        # (batch_size, sequence_length)
        cliprange: float,                   # 裁剪范围 ε
) -> Tuple[torch.Tensor, Dict[str, torch.Tensor]]:

**实现：**

In [ ]:
A = advantages.to(dtype=policy_log_probs.dtype, device=policy_log_probs.device)
log_ratio = policy_log_probs - old_log_probs
ratio = torch.exp(log_ratio)
clipped_ratio = torch.clamp(ratio, 1.0 - cliprange, 1.0 + cliprange)

unclipped_obj = ratio * A
clipped_obj = clipped_ratio * A
loss = -torch.minimum(unclipped_obj, clipped_obj)

# 统计裁剪比例
is_clipped = clipped_obj < unclipped_obj
metadata = {
    "ratio": ratio,
    "clipped_ratio": clipped_ratio,
    "is_clipped": is_clipped,
    "clip_fraction": is_clipped.float().mean()
}

### 策略梯度损失包装器

定义统一的损失函数接口，**支持的损失类型**包括：

- `"no_baseline"`：使用原始奖励作为优势
- `"reinforce_with_baseline"`：使用分组归一化优势
- `"grpo_clip"`：带裁剪的 GRPO 损失（离策略）
- `"grpo_no_clip"`：不带裁剪的 GRPO 损失

In [ ]:
def compute_policy_gradient_loss(
        policy_log_probs: torch.Tensor,
        loss_type: Literal["no_baseline", "reinforce_with_baseline", "grpo_clip", "grpo_no_clip"],
        raw_rewards: torch.Tensor | None = None,
        advantages: torch.Tensor | None = None,
        old_log_probs: torch.Tensor | None = None,
        cliprange: float | None = None,
) -> Tuple[torch.Tensor, Dict[str, torch.Tensor]]:

## 4.4 工具函数实现

### 掩码均值 (Masked Mean)

在响应token上计算平均值：


In [ ]:
def masked_mean(
        tensor: torch.Tensor,    # (batch_size, sequence_length)
        mask: torch.Tensor,      # (batch_size, sequence_length)
        dim: int | None = None,
) -> torch.Tensor:

**实现：**

In [ ]:
m = mask.to(device=tensor.device, dtype=tensor.dtype)
masked_sum = (tensor * m).sum() if dim is None else (tensor * m).sum(dim=dim)
denom = m.sum() if dim is None else m.sum(dim=dim)
return masked_sum / denom

### 掩码归一化 (Masked Normalize)

按常数归一化而不是按掩码元素数量：


In [ ]:
def masked_normalize(
        tensor: torch.Tensor,
        mask: torch.Tensor,
        normalize_constant: float,
        dim: int | None = None,
) -> torch.Tensor:

**实现：**


In [ ]:
m = mask.to(device=tensor.device, dtype=tensor.dtype)
masked_sum = (tensor * m).sum() if dim is None else (tensor * m).sum(dim=dim)
return masked_sum / float(normalize_constant)

## 4.5 GRPO 微批次训练步骤

### 核心函数

In [ ]:
def grpo_microbatch_train_step(
        policy_log_probs: torch.Tensor,         # (batch_size, sequence_length)
        response_mask: torch.Tensor,            # (batch_size, sequence_length)
        gradient_accumulation_steps: int,
        loss_type: str,
        raw_rewards: torch.Tensor | None = None,
        advantages: torch.Tensor | None = None,
        old_log_probs: torch.Tensor | None = None,
        cliprange: float | None = None,
        length_norm: str = "masked_mean",
) -> Tuple[torch.Tensor, Dict[str, torch.Tensor]]:

### 训练步骤详解

1. **计算每个token的损失**

In [ ]:
per_token_loss, meta = compute_policy_gradient_loss(
    policy_log_probs=policy_log_probs,
    loss_type=loss_type,
    raw_rewards=raw_rewards,
    advantages=advantages,
    old_log_probs=old_log_probs,
    cliprange=cliprange,
)

2. **长度归一化**

In [ ]:
mask = response_mask.to(dtype=per_token_loss.dtype, device=per_token_loss.device)
if length_norm == "masked_mean":
    per_example_loss = masked_mean(per_token_loss, mask, dim=1)  # (batch_size,)
elif length_norm == "masked_normalize":
    per_example_loss = masked_normalize(per_token_loss, mask, dim=1, constant_normalizer=max_len)

3. **批次平均和梯度累积**

In [ ]:
microbatch_loss = per_example_loss.mean()  # 标量
loss = microbatch_loss / float(gradient_accumulation_steps)
loss.backward()  # 只累积梯度

4. **元数据收集**

In [ ]:
out_meta = dict(meta)  # 复制底层元数据
out_meta.update({
    "microbatch_loss": microbatch_loss.detach(),
    "per_example_loss_mean": per_example_loss.detach().mean(),
    "per_example_loss_std": per_example_loss.detach().std(unbiased=False),
})

## 4.6 完整 GRPO 训练循环

由于 RL 训练需要消耗大量资源，为了在准确度和资源消耗之间取得平衡，我们在下面的 GRPO 训练使用了两张 GPU：
- **GPU 0**：策略模型训练
- **GPU 1**：vLLM 推理和评估

### 关键超参数

In [ ]:
# 核心配置
N_GRPO_STEPS = 200              # 总训练步数
ROLLOUT_BATCH_SIZE = 256        # rollout 批次大小
GROUP_SIZE = 8                  # 每个问题的响应数 G
TRAIN_BATCH_SIZE = 256          # 训练批次大小
EPOCHS_PER_ROLLOUT_BATCH = 1    # 每个rollout批次的训练轮数

# 优势计算
ADVANTAGE_EPS = 1e-6           # 防止除零的小常数
USE_STD_NORMALIZATION = True   # 是否按标准差归一化

# 损失函数
LOSS_TYPE = "reinforce_with_baseline"  # 损失类型
CLIPRANGE = 0.2                 # 裁剪范围（用于离策略）

# 训练优化
LR = 1e-5                       # 学习率
GRAD_ACCUM_STEPS = 128          # 梯度累积步数
MAX_GRAD_NORM = 1.0            # 梯度裁剪
SEED = 2026  

### 验证批次大小关系

In [ ]:
# 批次大小的数学关系
assert TRAIN_BATCH_SIZE % GRAD_ACCUM_STEPS == 0
micro_batch_size = TRAIN_BATCH_SIZE // GRAD_ACCUM_STEPS

assert ROLLOUT_BATCH_SIZE % GROUP_SIZE == 0
n_prompts_per_rollout = ROLLOUT_BATCH_SIZE // GROUP_SIZE

assert TRAIN_BATCH_SIZE >= GROUP_SIZE

### 采样参数配置

In [ ]:
# Rollout 采样参数
rollout_params = SamplingParams(
    temperature=1.0,      # 高温度鼓励多样性
    top_p=1.0,
    max_tokens=1024,
    min_tokens=4,         # 防止空响应
    stop=["</answer>"],
    include_stop_str_in_output=True,
    n=GROUP_SIZE         # 每个prompt的响应数
)

# 评估采样参数
eval_params = SamplingParams(
    temperature=0.0,      # 确定性评估
    top_p=1.0,
    max_tokens=1024,
    stop=["</answer>"],
    include_stop_str_in_output=True
)

### 训练循环实现

In [ ]:
def main():
    # 1. 初始化模型和数据
    policy = init_policy_model()
    llm = init_vllm()
    train_data, val_data = load_data()

    # 2. 训练循环
    for step in range(N_GRPO_STEPS):
        print(f"=== GRPO Step {step+1}/{N_GRPO_STEPS} ===")

        # A. 采样 prompts 批次
        batch_prompts = sample_prompts(train_data, n_prompts_per_rollout)

        # B. 更新 vLLM 权重
        load_policy_into_vllm(policy, llm)

        # C. 生成 rollouts
        outputs = llm.generate(batch_prompts, rollout_params)

        # D. 收集响应和计算奖励
        all_responses, all_rewards = collect_rollouts_and_rewards(outputs)

        # E. 计算分组归一化优势
        advantages, raw_rewards, _ = compute_group_normalized_rewards(
            reward_fn=r1_zero_reward_fn,
            rollout_responses=all_responses,
            repeated_ground_truths=all_rewards,  # GRPO 不需要 GT
            group_size=GROUP_SIZE,
            advantage_eps=ADVANTAGE_EPS,
            normalize_by_std=USE_STD_NORMALIZATION
        )

        # F. 创建训练数据集
        dataset = create_training_dataset(batch_prompts, all_responses, advantages)

        # G. 训练步骤
        train_on_dataset(policy, dataset, optimizer)

        # H. 评估
        if step % EVAL_EVERY_STEPS == 0:
            evaluate_policy(policy, llm, val_data)

# 第五部分：GRPO 实验


### 学习目标

在本部分中，我们将通过一系列系统性的实验来探索 GRPO 算法的各个方面，包括：
1. 超参数调优（学习率、基线、归一化等）
2. 算法变体比较（在策略 vs 离策略）
3. 消融实验（裁剪、提示等）
4. 最终的排行榜挑战

此部分的完整实现脚本见`grpo_experiments.py`。

## 5.1 实验基础设置

### 实验配置

所有 GRPO 实验的默认配置如下，除非在特定实验下有修改，否则均使用如下配置：

In [ ]:
@dataclass
class GRPOConfig:
    # Model & Data
    base_model: str = "/home/magnus-share/xuhu/model/Qwen2___5-Math-1___5B"
    train_data_path: str = "data/sft/sft_gpt-oss-120b_filtered.jsonl"
    val_data_path: str = "data/gsm8k/test.jsonl"
    output_dir: str = "results/grpo"
    prompt_template_path: str = "cs336_alignment/prompts/r1_zero.prompt"

    # GRPO Hyperparams
    n_grpo_steps: int = 200
    rollout_batch_size: int = 256
    group_size: int = 8
    train_batch_size: int = 256
    epochs_per_rollout_batch: int = 1
    advantage_eps: float = 1e-6
    cliprange: float = 0.2
    use_std_normalization: bool = False
    length_norm: str = "masked_mean"  # "masked_mean" or "masked_normalize"
    reward_fn: str=r1_zero_reward_fn     # r1_zero_reward_fn, question_only_reward_fn

    # Loss Type: "no_baseline", "reinforce_with_baseline", "grpo_clip", "grpo_no_clip"
    loss_type: str = "reinforce_with_baseline"

    # Training Hyperparams
    lr: float = 1e-5
    grad_accum_steps: int = 64
    max_grad_norm: float = 1.0
    seed: int = 2026

    # Evaluation
    eval_every_steps: int = 10
    eval_max_examples: int = 256  # max=1319

    # Devices
    device_policy: str = "cuda:1"
    device_eval: str = "cuda:0"

    # vLLM memory behavior
    vllm_gpu_memory_utilization: float = 0.7

    # Generation params
    gen_temperature: float = 1.0
    gen_top_p: float = 1.0
    gen_max_tokens: int = 1024
    gen_min_tokens: int = 4
    gen_stop: tuple = ("</answer>",)
    gen_include_stop_str_in_output: bool = True

    # Eval generation params
    eval_temperature: float = 0.0
    eval_top_p: float = 1.0
    eval_max_tokens: int = 1024
    eval_stop: tuple = ("</answer>",)
    eval_include_stop_str_in_output: bool = True

    # Optional knobs
    wandb_project: str = "cs336-a5-grpo"
    run_name: Optional[str] = None  # set to a string if you want

## 5.2 学习率调整实验 (grpo_learning_rate)

学习率是 GRPO 训练中最关键的超参数之一。本实验旨在找到最优的学习率设置。

### 实验设置 


In [ ]:
LEARNING_RATES = [1e-6, 5e-6, 1e-5, 2e-5, 5e-5, 1e-4]

for lr in LEARNING_RATES:
    base_config = GRPOConfig(
        n_grpo_steps=50,  # Shorter for sweep
        lr=lr
    )

### 结果分析

<center align="center">
    <img src="images/grpo_lr_eval_acc.png" width="400"/>
</center>

学习率结果：
- 1e-6:  18.88% (收敛太慢)
- 5e-6:  32.98% (仍然偏慢)
- 1e-5:  51.71% (最佳性能)
- 2e-5:  66.57% (略有下降)
- 5e-5:  49.28% (开始不稳定)
- 1e-4:  0.61% 

**最优学习率**：2e-5，达到66.57%的准确率，后续实验将采用此参数值。

## 5.3 基线影响实验 (grpo_baselines)

继续使用上述超参数（除了调整后的学习率），我们现在将研究策略梯度损失对性能的影响。我们处于 `on-policy` 设置，因此我们将比较损失类型：

- `no_baseline`
- `reinforce_with_baseline`

### 实验设置

In [ ]:

# BASELINE_TYPES = ["no_baseline", "reinforce_with_baseline"]
BASELINE_TYPES = ["reinforce_with_baseline", "no_baseline"]

for loss_type in BASELINE_TYPES:
    config = GRPOConfig(
        n_grpo_steps=50,  # Shorter for sweep
        # rollout_batch_size=64, 
        loss_type=loss_type,
        lr=2e-5,  # Best from LR sweep
        use_std_normalization=True
    )

### 理论分析

**无基线 (no_baseline)**：使用原始奖励 $r(q,o)$ 作为优势，简单直接，但方差较大，对奖励尺度敏感

**有基线 (reinforce_with_baseline)**：使用归一化优势 $A^{i}$，降低梯度方差，提高稳定性，对奖励尺度不敏感

### 实验结果

<center class="half">
    <img src="images/grpo_baseline_train_loss.png" width="300"/>
    <img src="images/grpo_baseline_train_entropy.png" width="300"/>
    <img src="images/grpo_baseline_eval_acc.png" width="300"/>
</center>

基线实验结果：
- `no_baseline`:           49.68%
- `reinforce_with_baseline`: 34.95% 


我们发现`reinforce_with_baseline`损失下降的更快，但为什么在测试集上表现性能更差呢？

通过观察熵变化曲线我们发现`reinforce_with_baseline`的熵很快变得很低，意味着它过早地变得“非常自信”，停止了探索。它可能只是死记硬背了某种特定的输出模式（Mode Collapse），而不是真正学会了推理。而`no_baseline`保持了较高的熵，说明它在训练过程中一直在尝试不同的可能，保留了探索能力。虽然`no_baseline`格式学习稍慢，但它是伴随着验证集准确率提升而稳步学习的。

**结论**：在当前的超参和模型设置下，使用`no_baseline`是更优的选择

## 5.4 长度归一化实验 (grpo_length_normalization)

比较不同的长度归一化方法对 GRPO 训练的影响。

- `masked_mean`：对响应token求平均：`loss = sum(loss_per_token) / num_response_tokens`，短响应获得更多权重（因为除数小）

- `masked_normalize`：按固定常数归一化：`loss = sum(loss_per_token) / constant`，所有响应获得相等权重

### 实验设置

In [ ]:
NORMALIZATION_TYPES = ["masked_mean", "masked_normalize"]

for norm_type in NORMALIZATION_TYPES:
    config = GRPOConfig(
        n_grpo_steps=50,  # Shorter for sweep
        length_norm=norm_type,
        loss_type="reinforce_with_baseline",
        lr=2e-5
    )

### 实验结果

<center class="half">
    <img src="images/grpo_length_norm_train_loss.png" width="300"/>
    <img src="images/grpo_length_norm_train_entropy.png" width="300"/>
    <img src="images/grpo_length_norm_train_grad_norm.png" width="300"/>
    <img src="images/grpo_length_norm_eval_acc.png" width="300"/>
</center>


长度归一化实验结果：
- `masked_mean``:      69.37%
- `masked_normalize``: 63.38% (+1.4%)

仅做 $r - \mu$。如果 Reward 的数值范围波动较大，计算出的 Advantage 绝对值就会很大。这直接导致计算出的梯度（Gradient）非常大。随着训练进行，模型对某些样本越来越自信（或 Reward 差异变大），梯度范数不断膨胀。
蓝色 (Normalize): 做了 $\frac{r - \mu}{\sigma}$。无论 Reward 的原始数值是 0.1 还是 100，归一化后的 Advantage 通常都落在 $[-2, 2]$ 甚至更小的区间内。这相当于自适应地调整了梯度的尺度，保证了更新幅度的稳定。


`masked_normalize` 略优，因为它避免了短响应主导训练的问题，梯度范数更稳定（`masked_normalize` 的梯度更一致）

## 5.5 标准差归一化实验 (grpo_group_standard_deviation)

测试是否应该按组标准差进行优势归一化。

### 实验设置

In [ ]:
STD_SETTINGS = [True, False]

for use_std in STD_SETTINGS:
    config = GRPOConfig(
        use_std_normalization=use_std,
        length_norm="masked_normalize",
        loss_type="reinforce_with_baseline",
        n_grpo_steps=50,  # Shorter for sweep
        lr=2e-5,
        run_name=f"grpo_std_{use_std}",
        output_dir=f"results/grpo_std_{use_std}",
    )

### 实验结果

<center class="half">
    <img src="images/grpo_std_norm_train_loss.png" width="300"/>
    <img src="images/grpo_std_norm_train_entropy.png" width="300"/>
    <img src="images/grpo_std_norm_train_grad_norm.png" width="300"/>
    <img src="images/grpo_std_norm_eval_acc.png" width="300"/>
</center>


标准差归一化实验结果：
- `use_std=True`:   63.46%
- `use_std=False`:  68.00%


开启优势归一化 (use_std=True`) 在本实验中并没有带来性能提升，反而导致了严重的训练崩溃（Catastrophic Forgetting）。 这种崩溃极有可能是由于某些 Batch 中 Reward 的方差过小，导致归一化时除以了一个极小的数值，引发了梯度爆炸。

Advantage Normalization 的公式通常是：
            $$ \hat{A} = \frac{A - \mu}{\sigma + \epsilon} $$
如果某一个 Group 内的所有样本 Reward 非常接近（方差 $\sigma$ 极小）**，那么分母 $\sigma + \epsilon$ 就会非常小。这会导致计算出的优势 $\hat{A}$ 变得巨大（例如：$0.001 / 0.0001 = 10$）。巨大的 $\hat{A}$ 会直接乘在 Loss 中，导致巨大的梯度（Gradient Explosion）。

在 Step 100 左右，模型可能刚好遇到了一个 Batch，里面的生成结果 Reward 都差不多（比如都答对了，或者都答错了且错得一样），导致 $\sigma \approx 0$。归一化操作放大了这种微小的差异，导致了梯度爆炸。从图中我们也可以找到证据，在 Step 100 左右，梯度范数瞬间飙升。这说明模型进行了一次幅度极大的参数更新。紧接着梯度爆炸，Step 105 左右 Green 的熵（Entropy）突然飙升到 0.8 左右，高熵表示模型输出趋向于混乱。

## 5.6 off-policy 的 GRPO 实现 (grpo_off_policy)


在最基础的 GRPO 实现中，我们通常采用 on-policy 的训练模式。我们到目前为止实验的所有超参数都是 on-policy 的，即每个 rollout batch 只进行一次梯度更新。这种做法保证了训练的稳定性，严格遵循了策略梯度定理的假设，即用于计算梯度的数据必须来自当前正在被优化的策略本身。

在 LLM 训练中，生成数据（Rollout/Inference）的成本极其高昂。它需要模型逐个 Token 地进行推理，速度慢且消耗大量显存。相比之下，反向传播（训练）的速度要快得多。
如果每生成几百条数据，只更新一次参数就丢弃，这在计算资源上是极大的浪费。为了解决这个问题，我们现在将尝试 off-policy 训练，即每个 rollout batch 进行多次梯度更新（多 Epochs）。

> 模型生成一批新的回答，这个过程就叫 `Rollout`

为了实现 off-policy 训练，这里有几个参数需要重点关注：

- `rollout_batch_size`： 采样批次大小，
- `train_batch_size`：训练批次大小，每次梯度更新用多少样本（通常 ≤ rollout_batch_size）
- `epochs_per_rollout_batch`：每批 rollout 数据训练几轮（epoch），越大的 epochs_per_rollout_batch 表示越 off-policy

⚠️ 注意：epochs_per_rollout_batch 并不总是越大越好，过度复用可能导致`策略漂移（policy drift）`，即策略偏离太大、发散，进而导致训练崩溃。所以需要像 PPO 那样用 clip（裁剪） 或 KL penalty 来约束更新幅度。

### 1. epochs_per_rollout_batch

在 run_off_policy_sweep_experiment 函数中，你可以看到不同的配置：
```
OFF_POLICY_CONFIGS = [
    {"epochs_per_rollout_batch": 1, "train_batch_size": 256, "grad_accum_steps": 128, "name": "on_policy"},
    {"epochs_per_rollout_batch": 2, "train_batch_size": 128, "grad_accum_steps": 128, "name": "off_policy_2x"},
    {"epochs_per_rollout_batch": 4, "train_batch_size": 128, "grad_accum_steps": 64, "name": "off_policy_4x"},
    {"epochs_per_rollout_batch": 8, "train_batch_size": 128, "grad_accum_steps": 32, "name": "off_policy_8x"},
]
```

其中核心控制参数 epochs_per_rollout_batch 是开启 Off-Policy 的“开关”。

在主循环中：

In [ ]:
# 外层循环控制这批数据被复用的次数
for epoch in range(cfg.epochs_per_rollout_batch):
    random.shuffle(dataset_indices) # 打乱数据
    # ... 进行多次梯度下降 ...

当 epoch > 0 时，模型参数 $\theta$ 已经更新了，但数据还是第 0 轮生成的。此时 $\pi_\theta \neq \pi_{old}$，这就构成了 Off-Policy 训练。

### 2. 固定参考概率：计算 old_log_probs

为了计算重要性采样比率 $r_t(\theta) = \frac{\pi_\theta(a_t|s_t)}{\pi_{old}(a_t|s_t)}$，我们必须记住数据生成时的概率 $\pi_{old}$。

In [ ]:
# 如果是 Off-Policy (grpo_clip)，必须先计算旧概率
if cfg.loss_type in ("grpo_clip", "grpo_no_clip"):
    print("Computing old logprobs...")
    policy.eval() # 冻结模型
    
    with torch.no_grad():
        # ... 遍历所有生成的 response ...
        out = get_response_log_probs(policy, input_ids, labels)
        old_log_probs_list.append(out["log_probs"].cpu())
    
    # 将这些概率拼接并保存起来，作为后续计算 ratio 的分母
    old_log_probs = torch.cat(padded, dim=0)

这一步至关重要。因为 vLLM 生成文本时通常只返回文本，不返回 Token 级别的 LogProbs。所以代码在开始训练循环前，先用当前的 Policy 跑一遍推理，把所有 Token 的 LogProb 存下来作为“基准”。在接下来的 epochs 循环中，这个 old_log_probs 保持不变。

### 3. 损失函数计算：PPO Clipping


在训练代码的内部循环中，代码将当前的 LogProbs 和保存的 Old LogProbs 同时传给损失函数：



In [ ]:
# 获取当前参数下的概率 (分子)
out = get_response_log_probs(policy, input_ids, labels, ...)
policy_log_probs = out["log_probs"]

# 获取生成时的概率 (分母)
batch_old_log_probs = old_log_probs[indices][:, :T]

# 计算 Loss
loss, loss_meta = grpo_microbatch_train_step(
    policy_log_probs=policy_log_probs,
    old_log_probs=batch_old_log_probs, # 传入旧概率
    loss_type=cfg.loss_type,           # 这里通常是 "grpo_clip"
    cliprange=cfg.cliprange,           # 例如 0.2
    ...
)

在标准的 On-Policy（如原始 REINFORCE）中，数据用一次就丢。而在 Off-Policy 设置下，随着参数更新，当前的策略 $\pi_\theta$ 已经偏离了生成数据时的策略 $\pi_{old}$，因此必须引入`重要性采样（Importance Sampling）` 和`裁剪（Clipping）` 机制来修正偏差。

在 `gpro_helper.py` 中，实现了 Off-Policy 的核心函数 compute_grpo_clip_loss，其实现了`重要性采样（Importance Sampling）` 和`裁剪（Clipping）` 机制。

In [ ]:
def compute_grpo_clip_loss(
        advantages: torch.Tensor,
        policy_log_probs: torch.Tensor, # 当前正在更新的策略 π_theta 的 LogProbs
        old_log_probs: torch.Tensor,    # 生成数据时的旧策略 π_old 的 LogProbs (固定不动)
        cliprange: float,               # 裁剪范围 ε (例如 0.2)
        loss_type: Literal["grpo_clip", "grpo_no_clip"] = "grpo_clip",
) -> Tuple[torch.Tensor, Dict[str, torch.Tensor]]:
    
    # ... (省略形状检查) ...

    # 1. 计算重要性采样比率 (Importance Sampling Ratio)
    # r_t(θ) = π_theta(a_t | s_t) / π_old(a_t | s_t)
    # 在对数空间计算：log(a/b) = log(a) - log(b)
    log_ratio = policy_log_probs - old_log_probs
    ratio = torch.exp(log_ratio)  # 得到 r_t(θ)

    if loss_type == "grpo_no_clip":
        # 如果不裁剪，就是标准的 Off-Policy 策略梯度 (容易训练不稳定)
        loss = -(ratio * A) 
        return loss, metadata

    elif loss_type == "grpo_clip":
        # 2. 计算未裁剪的目标函数 (Surrogate 1)
        # L^CPI = r_t(θ) * A_t
        unclipped_obj = ratio * A

        # 3. 计算裁剪后的目标函数 (Surrogate 2)
        # clip(r_t(θ), 1-ε, 1+ε) * A_t
        clipped_ratio = torch.clamp(ratio, 1.0 - cliprange, 1.0 + cliprange)
        clipped_obj = clipped_ratio * A

        # 4. 取最小值 (PPO 的保守策略)
        # L^CLIP = min(r_t(θ)*A_t, clip(...)*A_t)
        min_obj = torch.minimum(unclipped_obj, clipped_obj)
        
        # 5. 取负号因为我们要最小化 Loss
        loss = -min_obj

        # ... (记录 metadata) ...
        return loss, metadata

## 5.7 off-policy GRPO 超参数扫描 (grpo_off_policy_sweep)

### 实验设计

扫描 off-policy 的关键超参数：

In [ ]:
OFF_POLICY_CONFIGS = [
    {"epochs_per_rollout_batch": 1, "train_batch_size": 256, "grad_accum_steps": 128, "name": "on_policy"},
    {"epochs_per_rollout_batch": 2, "train_batch_size": 128, "grad_accum_steps": 128, "name": "off_policy_2x"},
    {"epochs_per_rollout_batch": 4, "train_batch_size": 128, "grad_accum_steps": 64, "name": "off_policy_4x"},
    {"epochs_per_rollout_batch": 8, "train_batch_size": 128, "grad_accum_steps": 32, "name": "off_policy_8x"},
]

### 实验结果

<center class="half">
    <img src="images/grpo_off_policy_train_clip_fraction.png" width="300"/>
    <img src="images/grpo_off_policy_train_entropy.png" width="300"/>
    <img src="images/grpo_off_policy_eval_acc.png" width="300"/>
</center>

off-policy 扫描结果：
- `on_policy(1 epoch)`:    58.30%
- `off_policy_2x(2 epochs)`: 73.84% 
- `off_policy_4x(4 epochs)`: 68.84% 
- `off_policy_8x(8 epochs)`: 63.69% 

off_policy_8x 的高裁剪率说明当前的策略已经偏离旧策略太远了。裁剪机制一直在拼命工作以阻止过大的更新，但这同时也意味着大部分梯度信息被截断了，或者剩下的梯度估计非常不准确。这解释了为什么 8x 训练效果最差。Brown (8x) 的熵下降慢，说明模型迟迟无法确信某种策略。由于反复在陈旧的数据上强行更新，模型陷入了混乱，无法收敛到一个确定的、高奖励的分布上。

强行增加 Off-Policy 的程度（即增加重复训练次数）会导致严重的训练不稳定和性能崩溃，尤其是在高倍率（8x）下。轻微的 Off-Policy (2x) 表现最佳。这验证了强化学习中的一个基本原理：PPO/GRPO 是基于 Trust Region（信任域）的算法，如果旧策略（采样数据的策略）和新策略（当前训练的策略）差异过大（KL 散度过高），重要性采样比率（Importance Sampling Ratio）会失效，导致数值爆炸。


## 5.8 裁剪消融实验 (grpo_off_policy_clip_ablation)


测试裁剪机制是否真的必要。

### 实验设置

In [ ]:
CLIP_SETTINGS = ["grpo_clip", "grpo_no_clip"]

# Use best off-policy config
for loss_type in CLIP_SETTINGS:
    config = GRPOConfig(
        loss_type=loss_type,
        epochs_per_rollout_batch=2,
        train_batch_size=128,  # Fixed: 128 % 128 = 0
        grad_accum_steps=128,  # Ensure compatibility
        n_grpo_steps=50,  # Shorter for sweep
        use_std_normalization=False,
        length_norm="masked_normalize",
        lr=2e-5,
        eval_max_examples = 256,  # max=1319
    )


### 实验结果

<center class="half">
    <img src="images/grpo_clip_train_entropy.png" width="300"/>
    <img src="images/grpo_clip_train_grad_norm.png" width="300"/>
    <img src="images/grpo_clip_train_clip_fraction.png" width="300"/>
    <img src="images/grpo_clip_eval_acc.png" width="300"/>
</center>

裁剪消融实验结果：
- `grpo_clip`:     79.99%
- `grpo_no_clip`:  68.23%

**结论**：裁剪对于离策略训练至关重要，防止策略发散。没有 Clip 机制（Red），模型训练虽然初期看似正常，但最终遭遇了灾难性的梯度爆炸，导致训练彻底失败；而开启 Clip 机制（Blue）虽然初期经历了剧烈的震荡，但最终成功稳定下来，并取得了远超 No Clip (Red) 的性能。


## 5.9 提示消融实验 (grpo_prompt_ablation)


比较不同提示对 GRPO 性能的影响。


### 提示对比

**R1-Zero 提示**：
```
A conversation between User and Assistant...
User: {question}
Assistant: <think>
```

**Question-Only 提示**：
```
{question}
```

### 实验设置

In [ ]:
PROMPT_TYPES = ["r1_zero", "question_only"]

for prompt_type in PROMPT_TYPES:
    if prompt_type == "r1_zero":
        template_path = "cs336_alignment/prompts/r1_zero.prompt"
        reward_fn = r1_zero_reward_fn
    else:
        template_path = "cs336_alignment/prompts/question_only.prompt"
        reward_fn = question_only_reward_fn

    config = GRPOConfig(
        prompt_template_path=template_path,
        reward_fn=reward_fn,
        use_std_normalization=False,
        length_norm="masked_normalize",
        loss_type="grpo_clip",
        n_grpo_steps=50,  # Shorter for sweep
        lr=2e-5
    )

### 实验结果

<center class="half">
    <img src="images/grpo_prompt_train_loss.png" width="300"/>
    <img src="images/grpo_prompt_train_entropy.png" width="300"/>
    <img src="images/grpo_prompt_train_format_reward.png" width="300"/>
    <img src="images/grpo_prompt_train_acc_reward.png" width="300"/>
    <img src="images/grpo_prompt_eval_acc.png" width="300"/>
</center>

```
提示消融实验结果：
- `r1_zero`:      46.32%
- `question_only`:  71.72%
```

蓝色曲线的下跌说明模型在训练过程中逐渐“遗忘”了如何正确输出格式。在 GRPO/PPO 中，如果格式错误导致答案无法解析，通常会得到一个很低的 Reward。这导致模型陷入混乱。并且蓝色曲线在 Step 95-100 左右，发生了梯度爆炸。当模型在 r1_zero 的复杂约束下挣扎时，可能出现了某些极端的样本（例如格式极其错误但偶然蒙对了答案，或者格式正确但被判错），导致 Loss 变得极大。

#### 为什么 r1_zero 会失败？

- Cold Start 问题：DeepSeek-R1 论文中提到，纯 RL (R1-Zero) 在初期非常不稳定，且容易产生不可读的输出。如果你的基座模型没有经过针对 <think> 格式的 SFT (Cold Start Data)，直接上 RL 强行要求它输出这种格式，模型很难通过随机探索找到正确的路径。
- Prompt 与模型能力不匹配：question_only 表现好，说明模型本身是有知识的。r1_zero 强加了复杂的思维链格式，这增加了生成的长度和复杂性。如果模型本身上下文能力或指令遵循能力不足，这种复杂的 Prompt 反而成了干扰（Distraction）。
- Reward 设计缺陷：如果你的 Reward 函数对格式错误的惩罚不够平滑（例如：格式错直接给 -1，格式对给 +1），模型可能在探索过程中因为频繁遭遇“格式惩罚”而梯度爆炸。

## 5.10 排行榜挑战 (leaderboard)

### 主要发现

1. **学习率调优至关重要**：2e-5 是最佳选择，比默认值高一个数量级

2. **基线显著提升性能**：使用分组归一化优势比原始奖励好3-4个百分点

3. **长度归一化影响稳定性**：`masked_normalize` 比 `masked_mean` 更稳定

4. **标准差归一化可有可无**：不使用标准差归一化略优

5. **离策略大幅提升效率**：2倍的样本效率，比 on-policy 性能提升15个百分点

6. **裁剪防止发散**：在离策略设置中必不可少

7. **提示格式很重要**：在此实验配置下简单提示比结构化提示好25个百分点

> 注意：由于 RL 训练对超参数非常敏感，并且比较吃资源，我们的实验大多只跑了`n_grpo_steps=50`，最终的结果可能还没有稳定下来。例如，下图所示大部分的 SOTA 方法需要跑至少 2000 step 才能又一个比价好的结果，因此`主要发现`只能作为参考，更多 tricks 可以参考网上已有经验。

<center align="center">
    <img src="images/computational_cost_comparison.png" width="400"/>
</center>

### 挑战目标

在4小时训练时间内，使用2个H100 GPU，获得尽可能高的验证准确率。

这部分留待大家继续探索！期待大家多多分享自己跑出来的 SOTA 结果！